biblioteci necesare

In [1]:
import cv2 as cv
import numpy as np
import os
from copy import deepcopy

functie pentru a vizualiza imaginile

In [2]:
def show_image(title,image, fx=0.4, fy=0.4):
    image=cv.resize(image,(0,0),fx=fx,fy=fy)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

functie pentru a gasi parametrii ideali in separarea mesei de tabla (luata din laborator)

In [3]:
def find_color_values_using_trackbar(frame):

    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    def nothing(x):
        pass

    cv.namedWindow("Trackbar")
    cv.createTrackbar("LH", "Trackbar", 0, 255, nothing)
    cv.createTrackbar("LS", "Trackbar", 0, 255, nothing)
    cv.createTrackbar("LV", "Trackbar", 0, 255, nothing)
    cv.createTrackbar("UH", "Trackbar", 255, 255, nothing)
    cv.createTrackbar("US", "Trackbar", 255, 255, nothing)
    cv.createTrackbar("UV", "Trackbar", 255, 255, nothing)


    while True:

        l_h = cv.getTrackbarPos("LH", "Trackbar")
        l_s = cv.getTrackbarPos("LS", "Trackbar")
        l_v = cv.getTrackbarPos("LV", "Trackbar")
        u_h = cv.getTrackbarPos("UH", "Trackbar")
        u_s = cv.getTrackbarPos("US", "Trackbar")
        u_v = cv.getTrackbarPos("UV", "Trackbar")


        l = np.array([l_h, l_s, l_v])
        u = np.array([u_h, u_s, u_v])
        mask_table_hsv = cv.inRange(frame_hsv, l, u)

        res = cv.bitwise_and(frame, frame, mask=mask_table_hsv)
        cv.imshow("Frame", frame)
        cv.imshow("Mask", mask_table_hsv)
        cv.imshow("Res", res)


        if cv.waitKey(25) & 0xFF == ord('q'):
            print(f"Lower HSV: {l}, Upper HSV: {u}")
            break
    cv.destroyAllWindows()

functia prin care se extrage careul (tabla de joc)

In [4]:
def extrage_careu(image):
    #transformam imaginea in imagine HSV
    img_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    #vrem sa luam din imagine tabla albastra
    #valorile le-am gasit jucandu-ma cu functia din lab find_color_values_using_trackbar
    low_blue=(81, 0, 0)
    high_blue=(119, 255, 255)
    mask_blue_hsv = cv.inRange(img_hsv, low_blue,high_blue)
    #scapam de o parte din zgomot
    kernel = np.ones((3, 3), np.uint8)
    thresh = cv.erode(mask_blue_hsv, kernel)
    #show_image('image_thresholded',thresh)

    edges =  cv.Canny(thresh ,50,150)
    # show_image('edges',edges, 0.15,0.15)
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
    cv.imwrite('./decupaj.jpg',edges)

    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width = 2000
    height = 2000

    image_copy = image.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    # show_image("detected corners",image_copy,0.15,0.15)

    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")
    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)
    result = cv.warpPerspective(image, M, (width, height))
    # cv.imwrite('./decupaj.jpg',result)
    #scoatem tabla cea mica 
    width = 80 * 14
    height = 80 * 14
    top_right = (1738, 266)
    top_left = (264, 264)
    bottom_right = (1739, 1748)
    bottom_left = (257, 1741)
    

    puzzle = np.array([top_left, top_right, bottom_left, bottom_right], dtype="float32")
    destination_of_puzzle = np.array([[0,0], [width, 0], [0, height], [width, height]], dtype="float32")
    M = cv.getPerspectiveTransform(puzzle, destination_of_puzzle)
    result = cv.warpPerspective(result, M, (width, height))

    return result

In [5]:
img = cv.imread("./antrenare/1_01.jpg")
extrage_careu(img)

array([[[232, 201, 147],
        [221, 202, 155],
        [203, 199, 157],
        ...,
        [110,  87,  53],
        [122,  87,  43],
        [135,  89,  35]],

       [[228, 199, 144],
        [214, 199, 155],
        [199, 202, 166],
        ...,
        [110, 105,  79],
        [108,  86,  47],
        [121,  84,  34]],

       [[224, 197, 141],
        [208, 198, 156],
        [194, 207, 175],
        ...,
        [125, 141, 123],
        [104,  99,  67],
        [111,  84,  39]],

       ...,

       [[139,  86,  29],
        [121,  87,  41],
        [117, 108,  74],
        ...,
        [ 85,  98,  81],
        [ 84,  72,  41],
        [107,  75,  31]],

       [[145,  88,  28],
        [127,  86,  35],
        [112,  94,  54],
        ...,
        [ 82,  78,  50],
        [ 98,  73,  33],
        [118,  76,  25]],

       [[149,  88,  25],
        [135,  88,  32],
        [118,  90,  42],
        ...,
        [ 94,  73,  34],
        [113,  76,  27],
        [126,  76,  19]]

construim liniile orizontale si verticale pentru a separa casutele din tabla

In [6]:

lines_horizontal = [i * 80 for i in range(15)]
lines_vertical = [i * 80 for i in range(15)]

functia returneaza pozitia pesei nou adaugata

In [7]:
def determinare_poz(prev_table,curr_table, prev_configuration):
    #transformam imaginile in gray scale
    grey_prev_table = cv.cvtColor(prev_table,cv.COLOR_BGR2GRAY)
    grey_curr_table = cv.cvtColor(curr_table,cv.COLOR_BGR2GRAY)
    #le convertim la int 32
    img_prev = np.array(grey_prev_table, dtype=np.int32)
    img_curr = np.array(grey_curr_table, dtype=np.int32)

    #facem diferenta intre ele
    diff = np.abs(img_prev-img_curr)
    tile_poz=()

    #casutele trebuie taiate un pic pentru ca au un spatiu albastru intre ele care nu ar trebui sa intre in calcul
    tile_padd=20
    #daca pe casuta respectiva avg este mai mare ca aceasta valoare putem sa o consideram o casuta pe care s-a pus o piese
    thresh=30
    max_patch=0
    for i in range(14):
        for j in range(14):
            if prev_configuration[i][j]==' ':
                patch = diff[lines_horizontal[i]+tile_padd: lines_horizontal[i+1]-tile_padd, lines_vertical[j]+tile_padd: lines_vertical[j+1]-tile_padd]

                mean = np.mean(patch)
                if mean > thresh and mean > max_patch:
                    max_patch=mean
                    tile_poz= (i,j)

    return (tile_poz,curr_table)

lista cu toate numerele jucate in mathable

In [8]:
list_numbers=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,24,25,27,28,30,32,35,36,40,42,45,48,49,50,54,56,60,63,64,70,72,80,81,90]

functie care clasifica cifrele din fiecare patratel (luata din laborator)

In [9]:
def clasifica_cifra(poz_patch, curr_table):
    maxi=-np.inf
    poz=-1
    #luam casuta corespunzatoare cifrei pe care vrem sa o identificam
    patch= curr_table[lines_horizontal[poz_patch[0]]: lines_horizontal[poz_patch[0]+1], lines_vertical[poz_patch[1]]: lines_vertical[poz_patch[1]+1]]
    #o transformam ca pe templaturi in imagini alb-negru
    kernel = np.ones((2, 2), np.uint8)
    _, thresh = cv.threshold(cv.cvtColor(patch, cv.COLOR_BGR2GRAY), 100, 255, cv.THRESH_BINARY_INV)
    dilated = cv.erode(thresh, kernel)
    dilated = cv.dilate(dilated, kernel)
    
    #incercam toate templaturile si alegem cel care da scorul cel mai bun
    for j in list_numbers:
        img_template=cv.imread('./templates/'+str(j)+'.jpg')
        img_template= cv.cvtColor(img_template,cv.COLOR_BGR2GRAY)
        corr = cv.matchTemplate(dilated,img_template,  cv.TM_CCOEFF_NORMED)
        corr=np.max(corr)
        if corr>maxi:
            maxi=corr
            poz=j

    return poz

functie prin care aflam ce jucator incepe, respectiv rundele alternative ale celor doi jucatori

In [10]:
def get_turn_list(i,fisier):
    with open(fisier+ str(i)+ '_turns.txt', "r") as txt_file:
        content = txt_file.read()
    text=content.strip().split()
    player=text[0]
    lista=[int(text[x]) for x in range(1,len(text),2)]
    lista.append(51)
    return (lista,player)


hardcodam tabla goala de joc pentru a putea afla pozitiile speciale

In [11]:
n=14
game_table = [[' ' for _ in range(14)] for _ in range(14)]
#adaugam tot ce este simetric pentru usurinta
simboluri = [
    (1, 4, '%'),
    (2, 5, '-'),
    (3, 6, '+'),
    (4, 6, '*'),
    (4, 1, '%'),
    (5, 2, '-'),
    (6, 3, '*'),
    (6, 4, '+'),
    (0, 0, '3'),
    (0, 6, '3'),
    (6, 0, '3'),
    (1, 1, '2'),
    (2, 2, '2'),
    (3, 3, '2'),
    (4, 4, '2'),
]
#matricea este in mare parte simentrica
for x, y, simbol in simboluri:
    game_table[x][y] = simbol
    game_table[x][n - y - 1] = simbol
    game_table[n - x - 1][y] = simbol
    game_table[n - x - 1][n - y - 1] = simbol

#corectam unde matricea nu este simentrica
game_table[3][7]='*'
game_table[4][7]='+'
game_table[7][3]='+'
game_table[7][4]='*'
game_table[6][9]='*'
game_table[6][10]='+'
game_table[9][6]='+'
game_table[10][6]='*'

for linie in game_table:
    print(' '.join(linie))


3           3 3           3
  2     %         %     2  
    2     -     -     2    
      2     + *     2      
  %     2   * +   2     %  
    -                 -    
3     * +         * +     3
3     + *         + *     3
    -                 -    
  %     2   + *   2     %  
      2     * +     2      
    2     -     -     2    
  2     %         %     2  
3           3 3           3


functie div alterata pentru a respinge impartirea la 0 sau a numarului mai mic la cel mai mare

In [12]:
def fix_div(a, b):
    if b == 0 or a == 0:
        return -1
    else:
        nr1=max(a,b)
        nr2=min(a,b)
        if nr1%nr2==0:
            return nr1//nr2
        else:
            return -1


functiile **down_conf**, **up_conf**, **left_conf**, **right_conf** sunt construite pentru 
a verifica toate posibilitatile posibile de a obtine scorul 

In [13]:
def down_conf(poz_i,poz_j,numar,board_conf):

    if poz_i+2<14 and board_conf[poz_i+1][poz_j]!=' ' and board_conf[poz_i+2][poz_j]!=' ':
        cifra1_down= int(board_conf[poz_i+1][poz_j])
        cifra2_down= int(board_conf[poz_i+2][poz_j])
        if game_table[poz_i][poz_j]=='+':
            if cifra1_down+cifra2_down==numar:
                return True
            else:
                return False
        elif game_table[poz_i][poz_j]=='*':
            if cifra1_down*cifra2_down==numar:
                return True
            else:
                return False
        elif game_table[poz_i][poz_j]=='-':
            if abs(cifra1_down-cifra2_down)==numar:
                return True
            else:
                return False
        elif game_table[poz_i][poz_j]=='%':
            if fix_div(cifra1_down,cifra2_down)==numar:
                return True
            else:
                return False
        elif cifra1_down+cifra2_down==numar or abs(cifra1_down-cifra2_down)==numar or cifra1_down*cifra2_down==numar or fix_div(cifra1_down,cifra2_down)==numar:
            return True
        else:
            return False
    else:
        return False


In [14]:
def up_conf(poz_i,poz_j,numar,board_conf):
    if poz_i-2<14 and board_conf[poz_i-1][poz_j]!= ' ' and board_conf[poz_i-2][poz_j]!=' ':
        cifra1_up= int(board_conf[poz_i-1][poz_j])
        cifra2_up= int(board_conf[poz_i-2][poz_j])
       
        if game_table[poz_i][poz_j]=='+':
            if cifra1_up+cifra2_up==numar:
                return True
            else:
                return False
        elif game_table[poz_i][poz_j]=='*':
            if cifra1_up*cifra2_up==numar:
                return True
            else:
                return False
        elif game_table[poz_i][poz_j]=='-':
            if abs(cifra1_up-cifra2_up)==numar:
                return True
            else:
                return False
        elif game_table[poz_i][poz_j]=='%':
            if fix_div(cifra1_up,cifra2_up)==numar:
                return True
            else:
                return False
        elif cifra1_up+cifra2_up==numar or abs(cifra1_up-cifra2_up)==numar or cifra1_up*cifra2_up==numar or fix_div(cifra1_up,cifra2_up)==numar:
            return True
        else:
            return False
    else:
        return False


In [15]:
def left_conf(poz_i,poz_j,numar,board_conf):
    if poz_j-2<14 and board_conf[poz_i][poz_j-1]!=' ' and board_conf[poz_i][poz_j-2]!=' ':
        cifra1_left= int(board_conf[poz_i][poz_j-1])
        cifra2_left= int(board_conf[poz_i][poz_j-2])
        if game_table[poz_i][poz_j]=='+':
            if cifra1_left+cifra2_left==numar:
                return True
            else:
                return False
        elif game_table[poz_i][poz_j]=='*':
            if cifra1_left*cifra2_left==numar:
                return True
            else:
                return False
        elif game_table[poz_i][poz_j]=='-':
            if abs(cifra1_left-cifra2_left)==numar:
                return True
            else:
                return False
        elif game_table[poz_i][poz_j]=='%':
            if fix_div(cifra1_left,cifra2_left)==numar:
                return True
            else:
                return False
        elif cifra1_left+cifra2_left==numar or abs(cifra1_left-cifra2_left)==numar or cifra1_left*cifra2_left==numar or fix_div(cifra1_left,cifra2_left)==numar:
            return True
        else:
            return False
    else:
        return False


In [16]:
def right_conf(poz_i,poz_j,numar,board_conf):
    if poz_j+2<14 and board_conf[poz_i][poz_j+1]!=' ' and board_conf[poz_i][poz_j+2]!=' ':
        cifra1_right= int(board_conf[poz_i][poz_j+1])
        cifra2_right= int(board_conf[poz_i][poz_j+2])
        if game_table[poz_i][poz_j]=='+':
            if cifra1_right+cifra2_right==numar:
                return True
            else:
                return False
        elif game_table[poz_i][poz_j]=='*':
            if cifra1_right*cifra2_right==numar:
                return True
            else:
                return False
        elif game_table[poz_i][poz_j]=='-':
            if abs(cifra1_right-cifra2_right)==numar:
                return True
            else:
                return False
        elif game_table[poz_i][poz_j]=='%':
            if fix_div(cifra1_right,cifra2_right)==numar:
                return True
            else:
                return False
        elif cifra1_right+cifra2_right==numar or abs(cifra1_right-cifra2_right)==numar or cifra1_right*cifra2_right==numar or fix_div(cifra1_right,cifra2_right)==numar:
            return True
        else:
            return False
    else:
        return False


functie pentru a determina daca piesa lipita configuratiei se afla pe scor dublu sau triplu sau pe o pozitie normala

In [17]:
def poz_speciala(poz_i,poz_j):
    if game_table[poz_i][poz_j]!=' ' and game_table[poz_i][poz_j]!='*' and game_table[poz_i][poz_j]!='%' and game_table[poz_i][poz_j]!='-' and game_table[poz_i][poz_j]!='+':
        return (int(game_table[poz_i][poz_j]))
    return 0

Principala logica a programului

In [18]:
#prima tabla este o tabla goala cu cele 4 numere initiale puse in mijlocul ei
#acesata tabla urmeaza a fi prelucrata pentru fiecare joc
first_board_conf=[[' ' for _ in range(15)] for _ in range(15)]
first_board_conf[6][6]='1'
first_board_conf[6][7]='2'
first_board_conf[7][6]='3'
first_board_conf[7][7]='4'

first_board_img= cv.imread('./imagini_auxiliare/01.jpg')
first_table=extrage_careu(first_board_img)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#aici se schimba fisierul si numarul de jocuri
fisier_date='./antrenare/'
game_numbers=4
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
files=os.listdir(fisier_date)
files.sort()
board_conf=first_board_conf
first_img=first_table
cifre=[]
tile_poz=()
os.makedirs("351_Pirvulescu_Daria",exist_ok=True)
score_player1=0
score_player2=0
for i in range(1,game_numbers+1):
    #incepe un nou joc => toate variabilele trebuie resetate
    board_conf = deepcopy(first_board_conf)
    first_img = first_table
    tile_poz=()
    score_player1=0
    score_player2=0
    turns,player=get_turn_list(i,fisier_date)
    #stabilim daca e player 1 care primeste punctajul sau player2
    if player=='Player1':
        turn_player1=1
    else:
        turn_player1=0
    t=1
    
    scores_name = f'./351_Pirvulescu_Daria/{i}_scores.txt'
   
    for file in files:
        first_letter = os.path.basename(file)[0]
        if first_letter==str(i) :
            # suntem in jocul curent
            if file[-3:] == 'jpg':
               # daca randul a ajuns sa fie egal cu o imagine prelucrata inseamna ca playeri se schimba intre ei
                if int(file[2:4])==turns[t]:
                    #scriem scorulile
                    if turn_player1:
                        with open(scores_name, "a") as file_scores:
                            file_scores.write("Player1 "+ str(turns[t-1]) + " "+ str(score_player1)+"\n")
                    else:
                        with open(scores_name, "a") as file_scores:
                            file_scores.write("Player2 "+ str(turns[t-1]) + " "+ str(score_player2)+ "\n")

                    #se schimba playerul
                    turn_player1=1-turn_player1
                    if turn_player1:
                        score_player1=0
                    else:
                        score_player2=0
                    #avansam in turns
                    t+=1

                img = cv.imread(fisier_date + file)
                second_img = extrage_careu(img)
                #avem in tile_poz pozitia pe care s-a pus cifra curenta si in img patch-ul cu cifra curenta
                tile_poz,img = determinare_poz(first_img, second_img, board_conf)
                poz_i=tile_poz[0]
                poz_j=tile_poz[1]
                first_img = second_img
                #in cifra1 tinem cifra prezisa de functia noastra( de fapt se prezice un numar dar am lasat asa:)
                cifra1 = clasifica_cifra(tile_poz,img)
                board_conf[poz_i][poz_j] = str(cifra1)
                #avem datele necesare pt a scrie in fisiere
                tmp_name = f'./351_Pirvulescu_Daria/{file[0:4]}.txt'
                with open(tmp_name, "w") as file_poz_nr:  
                    file_poz_nr.write(str(poz_i+1)+chr(ord('@')+poz_j+1)+ " "+ str(cifra1))
                print(file)
                if turn_player1:
                    #joaca player 1
                    points_up = up_conf(poz_i,poz_j,cifra1,board_conf)
                    points_down = down_conf(poz_i,poz_j,cifra1,board_conf)
                    points_left = left_conf(poz_i,poz_j,cifra1,board_conf)
                    points_right = right_conf(poz_i,poz_j,cifra1,board_conf)
                    #adunam cate configuratii valide are 
                    nr = len([y for y in [points_up, points_down, points_left, points_right] if y == True])
                    nr_special= poz_speciala(poz_i,poz_j)
                    if nr_special!=0:
                        score_player1+=(cifra1*nr)*nr_special
                    else:
                        score_player1 += cifra1*nr
                else:
                    # joaca player 2
                    points_up = up_conf(poz_i,poz_j,cifra1,board_conf)
                    points_down = down_conf(poz_i,poz_j,cifra1,board_conf)
                    points_left = left_conf(poz_i,poz_j,cifra1,board_conf)
                    points_right = right_conf(poz_i,poz_j,cifra1,board_conf)
                    nr = len([y for y in [points_up, points_down, points_left, points_right] if y == True])
                    nr_special= poz_speciala(poz_i,poz_j)
                    if nr_special!=0:
                        score_player2+=(cifra1*nr)*nr_special
                    else:
                        score_player2 += cifra1*nr

    if turn_player1:
        with open(scores_name, "a") as file_scores:
            file_scores.write("Player1 "+ str(turns[t-1]) + " "+ str(score_player1))
    else:
        with open(scores_name, "a") as file_scores:
            file_scores.write("Player2 "+ str(turns[t-1]) + " "+ str(score_player2))


1_01.jpg
1_02.jpg
1_03.jpg
1_04.jpg
1_05.jpg
1_06.jpg
1_07.jpg
1_08.jpg
1_09.jpg
1_10.jpg
1_11.jpg
1_12.jpg
1_13.jpg
1_14.jpg
1_15.jpg
1_16.jpg
1_17.jpg
1_18.jpg
1_19.jpg
1_20.jpg
1_21.jpg
1_22.jpg
1_23.jpg
1_24.jpg
1_25.jpg
1_26.jpg
1_27.jpg
1_28.jpg
1_29.jpg
1_30.jpg
1_31.jpg
1_32.jpg
1_33.jpg
1_34.jpg
1_35.jpg
1_36.jpg
1_37.jpg
1_38.jpg
1_39.jpg
1_40.jpg
1_41.jpg
1_42.jpg
1_43.jpg
1_44.jpg
1_45.jpg
1_46.jpg
1_47.jpg
1_48.jpg
1_49.jpg
1_50.jpg
2_01.jpg
2_02.jpg
2_03.jpg
2_04.jpg
2_05.jpg
2_06.jpg
2_07.jpg
2_08.jpg
2_09.jpg
2_10.jpg
2_11.jpg
2_12.jpg
2_13.jpg
2_14.jpg
2_15.jpg
2_16.jpg
2_17.jpg
2_18.jpg
2_19.jpg
2_20.jpg
2_21.jpg
2_22.jpg
2_23.jpg
2_24.jpg
2_25.jpg
2_26.jpg
2_27.jpg
2_28.jpg
2_29.jpg
2_30.jpg
2_31.jpg
2_32.jpg
2_33.jpg
2_34.jpg
2_35.jpg
2_36.jpg
2_37.jpg
2_38.jpg
2_39.jpg
2_40.jpg
2_41.jpg
2_42.jpg
2_43.jpg
2_44.jpg
2_45.jpg
2_46.jpg
2_47.jpg
2_48.jpg
2_49.jpg
2_50.jpg
3_01.jpg
3_02.jpg
3_03.jpg
3_04.jpg
3_05.jpg
3_06.jpg
3_07.jpg
3_08.jpg
3_09.jpg
3_10.jpg
3_11.jpg
3